In [1]:
import torch
from divide_images import test_train
from dataloader import create_dataloader
import torchvision
import numpy as np
import matplotlib.pyplot as plt

## Splitting the mask dataset

In [2]:
src_folder = "./data/"
test_path = "./test_data/"
train_path = "./train_data/"

In [3]:
test_train(src_folder, test_path, train_path)

## Creating Data Loader

In [4]:
train_loader = create_dataloader(train_path)
test_loader = create_dataloader(test_path)

In [ ]:
classes = ('without mask', 'with mask')

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))